In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd.gradcheck import zero_gradients
from torch.autograd import Variable
import numpy as np
import time
import argparse
from torchsummary import summary
import torchvision
from scipy.special import softmax
import collections
import json 

from utils.mnist_models import cnn_3l_bn
from utils.resnet_cifar import resnet
from utils.test_utils import test, robust_test, robust_test_hybrid
from utils.data_utils import load_dataset, load_dataset_custom
from utils.io_utils import init_dirs, model_naming, test_file_save_name, test_probs_save_name
from utils.attack_utils import cal_loss, generate_target_label_tensor, pgd_attack, pgd_l2_attack, hybrid_attack
from utils.data_utils import load_dataset_tensor

ModuleNotFoundError: No module named '__main__.attack_utils'; '__main__' is not a package

In [ ]:
def robust_test(model, loss_fn, loader, args, att_dir, epoch=0, training_output_dir_name=None, 
                figure_dir_name=None, n_batches=0, train_data=False, 
                training_time=False):
    """
    n_batches (int): Number of batches for evaluation.
    """
    model.eval()
    num_correct, num_correct_adv, num_samples = 0, 0, 0
    steps = 1
    losses_adv = []
    losses_ben = []
    prob_dict = {}
    adv_examples = []
    adv_preds = []
    true_labels = []

    for t, (x, y, idx, ez, m) in enumerate(loader):
        x = x.cuda()
        y = y.cuda()
        x_var = Variable(x, requires_grad= True)
        y_var = Variable(y, requires_grad=False)
        if att_dir['targeted']:
            y_target = generate_target_label_tensor(
                               y_var.cpu(), args).cuda()
        else:
            y_target = y_var
        if 'PGD_linf' in att_dir['attack']:
            adv_x = pgd_attack(model, x, x_var, y_target, att_dir['attack_iter'],
                           att_dir['epsilon'], att_dir['eps_step'], att_dir['clip_min'],
                           att_dir['clip_max'], att_dir['targeted'], att_dir['rand_init'])
        elif 'PGD_l2' in att_dir['attack']:
            adv_x = pgd_l2_attack(model, x, x_var, y_target, att_dir['attack_iter'],
                           att_dir['epsilon'], att_dir['eps_step'], att_dir['clip_min'],
                           att_dir['clip_max'], att_dir['targeted'], att_dir['rand_init'], 
                           att_dir['num_restarts'])
        adv_examples.append(adv_x)
        # Predictions
        scores = model(x.cuda()) 
        _, preds = scores.data.max(1)
        scores_adv = model(adv_x)
        _, preds_adv = scores_adv.data.max(1)
        adv_preds.append(preds_adv)
        true_labels.append(y)
        # Losses
        batch_loss_adv = loss_fn(scores_adv, y)
        loss_adv = torch.mean(batch_loss_adv)
        losses_adv.append(loss_adv.data.cpu().numpy())
        batch_loss_ben = loss_fn(scores, y)
        loss_ben = torch.mean(batch_loss_ben)
        losses_ben.append(loss_ben.data.cpu().numpy())
        # Correct count
        num_correct += (preds == y).sum()
        num_correct_adv += (preds_adv == y).sum()
        num_samples += len(preds)
        
        if not training_time:
          if args.viz and steps == 1:
              if not os.path.exists(figure_dir_name):
                os.makedirs(figure_dir_name)
              custom_save_image(adv_x, preds_adv, y, args, figure_dir_name, 
                                train_data)
        
        if n_batches > 0 and steps==n_batches:
            break
        steps += 1

    acc = float(num_correct) / num_samples
    acc_adv = float(num_correct_adv) / num_samples
    print('Clean accuracy: {:.2f}% ({}/{})'.format(
        100.*acc,
        num_correct,
        num_samples,
    ))
    print('Adversarial accuracy: {:.2f}% ({}/{})'.format(
        100.*acc_adv,
        num_correct_adv,
        num_samples,
    ))

    return 100.*acc, 100.*acc_adv, np.mean(losses_ben), np.mean(losses_adv), adv_examples, adv_preds, true_labels

In [ ]:
torch.random.manual_seed(7)

parser = argparse.ArgumentParser()
# Data args
parser.add_argument('--dataset_in', type=str, default='MNIST')
parser.add_argument('--n_classes', type=int, default=10)
parser.add_argument('--num_samples', type=int, default=None)

# Model args
parser.add_argument('--model', type=str, default='cnn_3l',
                    choices=['resnet','wrn', 'cnn_3l', 'cnn_3l_bn', 'dn'])
parser.add_argument('--conv_expand', type=int, default=1)
parser.add_argument('--fc_expand', type=int, default=1)
parser.add_argument('--depth', type=int, default=28)
parser.add_argument('--width', type=int, default=10)
parser.add_argument('--lr_schedule', type=str, default='linear0')
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--test_batch_size', type=int, default=128)
parser.add_argument('--learning_rate', type=float, default=0.1)
# parser.add_argument('--weight_decay', type=float, default=2e-4)

# Defense args
parser.add_argument('--is_adv', dest='is_adv', action='store_true')
parser.add_argument('--attack', type=str, default='PGD_l2',
                    choices=['PGD_l2', 'PGD_linf', 'PGD_l2_hybrid_seed', 'PGD_l2_hybrid_replace'])
parser.add_argument('--epsilon', type=float, default=8.0)
parser.add_argument('--attack_iter', type=int, default=10)
parser.add_argument('--gamma', type=float, default=1.0)
parser.add_argument('--eps_step', type=float, default=2.0)
parser.add_argument('--is_dropping', dest='dropping', action='store_true')
parser.add_argument('--rand_init', dest='rand_init', action='store_true')
parser.add_argument('--eps_schedule', type=int, default=0)
parser.add_argument('--num_restarts', type=int, default=1)
parser.add_argument('--marking_strat', type=str, default=None)
parser.add_argument('--matching_path', type=str, default='matchings')
parser.add_argument('--degree_path', type=str, default='graph_data/degree_results')
parser.add_argument("--norm", default='l2', help="norm to be used")
parser.add_argument('--drop_thresh', type=int, default=100)
parser.add_argument('--drop_eps',type=float, default=0.0)
parser.add_argument('--curriculum', type=str, default='all')
parser.add_argument('--loss_fn', type=str, default='CE')

# Attack args
parser.add_argument('--new_attack', type=str, default='PGD_l2',
                    choices=['PGD_l2', 'PGD_linf', 'PGD_l2_hybrid_seed', 'PGD_l2_hybrid_replace'])
parser.add_argument('--new_epsilon', type=float, default=2.0)
parser.add_argument('--new_attack_iter', type=int, default=20)
parser.add_argument('--new_gamma', type=float, default=1.0)
parser.add_argument('--targeted', dest='targeted', action='store_true')
parser.add_argument('--clip_min', type=float, default=0)
parser.add_argument('--clip_max', type=float, default=1.0)
parser.add_argument('--new_rand_init',
                    dest='new_rand_init', action='store_true')
parser.add_argument('--new_num_restarts', type=int, default=1)
parser.add_argument('--new_marking_strat', type=str, default=None)

# IO args
parser.add_argument('--last_epoch', type=int, default=0)
parser.add_argument('--checkpoint_path', type=str, 
                    default='/data/abhagoji/models')
parser.add_argument('--is_viz', dest='viz', action='store_true')

# Trial args
parser.add_argument('--num_of_trials', type=int, default=1)
parser.add_argument('--save_test', dest='save_test', action='store_true')
parser.add_argument('--track_hard', dest='track_hard', action='store_true')

if torch.cuda.is_available():
    print('CUDA enabled')
else:
    raise ValueError('Needs a working GPU!')

In [ ]:
eps=3.0

In [ ]:
args = parser.parse_args("--dataset=MNIST --n_classes=2 --num_samples=5000 --model=cnn_3l_bn --attack=PGD_l2 --epsilon={} --attack_iter=50 --gamma=2.5 --new_attack=PGD_l2 --new_epsilon={} --new_attack_iter=50 --new_gamma=2.5 --trial_num=1 --is_adv  --rand_init --new_rand_init".format(eps,eps).split())

In [ ]:
args = parser.parse_args()

if args.num_samples is None:
    args.num_samples = 'All'
if args.drop_eps==0:
    args.drop_eps=args.epsilon

args.checkpoint_path = 'trained_models'

args.eps_step = args.epsilon*args.gamma/args.attack_iter
args.new_eps_step = args.new_epsilon*args.gamma/args.new_attack_iter
attack_params = {'attack': args.new_attack, 'epsilon': args.new_epsilon, 
             'attack_iter': args.new_attack_iter, 'eps_step': args.new_eps_step,
             'targeted': args.targeted, 'clip_min': args.clip_min,
             'clip_max': args.clip_max,'rand_init': args.new_rand_init, 
             'num_restarts': args.new_num_restarts}

In [ ]:
args.trial_num = 1
model_dir_name, log_dir_name, figure_dir_name, _ = init_dirs(
    args, train=False)
_, model_name = model_naming(args)
print('Loading %s' % model_dir_name)

training_time = False

if args.n_classes != 10:
    loader_train, loader_test, data_details = load_dataset_custom(
        args, data_dir='data', training_time=training_time)
else:
    loader_train, loader_test, data_details = load_dataset(
        args, data_dir='data', training_time=training_time)

num_channels = data_details['n_channels']

if 'MNIST' in args.dataset_in:
    if 'cnn_3l_bn' in args.model:
        net = cnn_3l_bn(args.n_classes, args.conv_expand, args.fc_expand)

if 'linf' in args.attack:
    args.epsilon /= 255.
    args.eps_step /= 255.

print("Using batch size of {}".format(args.batch_size))

net.cuda()

criterion = nn.CrossEntropyLoss(reduction='none')

net.eval()
ckpt_path = 'checkpoint_' + str(args.last_epoch)
net.load_state_dict(torch.load(model_dir_name + ckpt_path))
if 'hybrid' in args.new_attack:
    print('Using attack {}'.format(args.new_attack))
    f_eval = robust_test_hybrid
else:
    print('Using attack {}'.format(args.new_attack))
    f_eval = robust_test
n_batches_eval = 0
print('Training data results')
acc_train, acc_train_adv, loss_train, loss_train_adv, adv_examples, adv_preds, true_labels = f_eval(net, 
    criterion, loader_train, args, attack_params, 0, None, 
    figure_dir_name, n_batches=n_batches_eval, train_data=True, training_time=training_time)
print('Test data results')
acc_test, acc_test_adv, loss_test, loss_test_adv, adv_examples, adv_preds, true_labels = f_eval(net, 
    criterion, loader_test, args, attack_params, 0, None, 
    figure_dir_name, n_batches=n_batches_eval, train_data=False, training_time=training_time)